<a href="https://colab.research.google.com/github/kevinmoreno1-ux/ENTREGAS/blob/main/99-modelo_solucion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Especificar las rutas a los archivos
train_file_path = '/content/drive/MyDrive/IA/SaberPro/train.csv'
test_file_path = '/content/drive/MyDrive/IA/SaberPro/test.csv'  # Asegúrate de que este archivo existe
output_file_path = '/content/drive/MyDrive/IA/SaberPro/submission_solucion.csv'  # Ruta para guardar resultados

print("=== MODELO DE PREDICCIÓN DE RENDIMIENTO ACADÉMICO ===")
print("=" * 70)

# ============================================================================
# PROCESAMIENTO DE DATOS DE ENTRENAMIENTO
# ============================================================================

print("Cargando datos de entrenamiento...")
try:
    train_data = pd.read_csv(train_file_path)
    print(f"✓ Datos de entrenamiento cargados: {train_data.shape}")
except Exception as e:
    print(f"✗ Error cargando train.csv: {e}")
    raise

# Limpieza inicial: eliminar columna duplicada si existe
if 'F_TIENEINTERNET.1' in train_data.columns:
    train_data = train_data.drop(columns=["F_TIENEINTERNET.1"])

# Separar características numéricas y categóricas
numeric_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()

# Excluir variable objetivo e ID
if 'RENDIMIENTO_GLOBAL' in numeric_cols:
    numeric_cols.remove('RENDIMIENTO_GLOBAL')
if 'RENDIMIENTO_GLOBAL' in categorical_cols:
    categorical_cols.remove('RENDIMIENTO_GLOBAL')
if 'ID' in numeric_cols:
    numeric_cols.remove('ID')
if 'ID' in categorical_cols:
    categorical_cols.remove('ID')

print(f"Columnas numéricas: {len(numeric_cols)}")
print(f"Columnas categóricas: {len(categorical_cols)}")

# Manejo de valores faltantes
print("Aplicando preprocesamiento...")
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

train_data[numeric_cols] = imputer_num.fit_transform(train_data[numeric_cols])
train_data[categorical_cols] = imputer_cat.fit_transform(train_data[categorical_cols])

# Codificación de variables categóricas
label_encoders = {}
for col in categorical_cols:
    try:
        le = LabelEncoder()
        train_data[col] = le.fit_transform(train_data[col].astype(str))
        label_encoders[col] = le
    except Exception as e:
        print(f"Error codificando {col}: {e}")
        train_data = train_data.drop(columns=[col])
        if col in categorical_cols:
            categorical_cols.remove(col)

# Preparar datos para entrenamiento
X_train = train_data.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y_train = train_data["RENDIMIENTO_GLOBAL"]

# Estandarización de características numéricas
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])



#Entrenamiento del modelo
print("Entrenando modelo Random Forest...")
#model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
#model.fit(X_train_scaled, y_train)

# Entrenamiento del modelo (Variando hiperparametros)

# print("Entrenando modelo Random Forest...")
# model = RandomForestClassifier(n_estimators=200,
#                               max_depth=20,
#                               min_samples_split=10,
#                               min_samples_leaf=5,
#                               class_weight='balanced',
#                               random_state=42,
#                               n_jobs=-1)

# model.fit(X_train_scaled, y_train)

# Entrenamiento del modelo (Variando hiperparametros)

#print("Entrenando modelo Random Forest...")
#model = RandomForestClassifier(
#    n_estimators=280,
#    max_depth=28,
#    min_samples_split=4,
#    min_samples_leaf=1,
#    max_features=0.5,
#    bootstrap=True,
#    class_weight='balanced',
#    random_state=42,
#    n_jobs=-1)

#model.fit(X_train_scaled, y_train)

model = RandomForestClassifier(
    n_estimators=150,
    max_depth=15,
    min_samples_split=3,
    min_samples_leaf=1,
    max_features=0.7,
    bootstrap=True,
    random_state=42,
    n_jobs=-1)


model.fit(X_train_scaled, y_train)

print("✓ Modelo entrenado exitosamente!")
print(f"✓ Tamaño del conjunto de entrenamiento: {X_train_scaled.shape}")

# ============================================================================
# PROCESAMIENTO DE DATOS DE PRUEBA Y PREDICCIÓN
# ============================================================================

print("\nCargando datos de prueba...")
try:
    test_data = pd.read_csv(test_file_path)
    print(f"✓ Datos de prueba cargados: {test_data.shape}")
except Exception as e:
    print(f"✗ Error cargando test.csv: {e}")
    raise

# Conservar IDs para la submission
test_ids = test_data['ID'].copy()

# Aplicar misma limpieza inicial
if 'F_TIENEINTERNET.1' in test_data.columns:
    test_data = test_data.drop(columns=["F_TIENEINTERNET.1"])

# Asegurar que las columnas coincidan con el entrenamiento
missing_cols = set(X_train.columns) - set(test_data.columns)
if missing_cols:
    print(f"✗ Columnas faltantes en test: {missing_cols}")
    raise ValueError("Faltan columnas en el conjunto de prueba")

extra_cols = set(test_data.columns) - set(X_train.columns) - {'ID'}
if extra_cols:
    print(f"⚠ Columnas adicionales en test: {extra_cols}")

# Reordenar columnas para coincidir con el entrenamiento
test_data = test_data[X_train.columns.tolist() + ['ID']]

# Aplicar imputación con los mismos parámetros de entrenamiento
test_data[numeric_cols] = imputer_num.transform(test_data[numeric_cols])
test_data[categorical_cols] = imputer_cat.transform(test_data[categorical_cols])

# Codificar variables categóricas con los mismos encoders
for col in categorical_cols:
    if col in label_encoders:
        # Manejar valores no vistos asignándoles un valor común
        unique_values = set(test_data[col].astype(str))
        unknown_values = unique_values - set(label_encoders[col].classes_)
        if unknown_values:
            print(f"⚠ Valores no vistos en {col}: {unknown_values}")
            # Asignar el valor más frecuente del entrenamiento
            most_frequent = label_encoders[col].classes_[0]
            test_data[col] = test_data[col].apply(
                lambda x: x if x in label_encoders[col].classes_ else most_frequent
            )
        test_data[col] = label_encoders[col].transform(test_data[col].astype(str))

# Estandarización con el scaler de entrenamiento
X_test_scaled = test_data.drop(columns=['ID'])
X_test_scaled[numeric_cols] = scaler.transform(X_test_scaled[numeric_cols])

# Realizar predicciones
print("Realizando predicciones...")
y_pred = model.predict(X_test_scaled)

# Crear archivo de submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': y_pred
})

submission.to_csv(output_file_path, index=False)
print(f"✓ Archivo de submission guardado en: {output_file_path}")
print(f"✓ Total de predicciones: {len(y_pred)}")
print("=" * 70)

Mounted at /content/drive
=== MODELO DE PREDICCIÓN DE RENDIMIENTO ACADÉMICO ===
Cargando datos de entrenamiento...
✓ Datos de entrenamiento cargados: (692500, 21)
Columnas numéricas: 5
Columnas categóricas: 13
Aplicando preprocesamiento...
Entrenando modelo Random Forest...
✓ Modelo entrenado exitosamente!
✓ Tamaño del conjunto de entrenamiento: (692500, 18)

Cargando datos de prueba...
✓ Datos de prueba cargados: (296786, 20)
⚠ Valores no vistos en E_PRGM_ACADEMICO: {'COMUNICACIÖN SOCIAL', 'AGROINDUSTRIA', 'LICENC.EN EDUCACION BASICA PRIMARIA ENF.EN CIENCIAS NATURALESEDUC.AMBIENTAL MATEMA.LENGUA CASTELLANA', 'LICENCIATURA EN MATEMATICA E INFORMATICA', 'NEGOCIOS INTERNACIONALES BILINGUE', 'LICENCIATURA EN EDUCACION ENFASIS EN HUMANIDADES', 'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN IDIOMAS EXTRANJEROS', 'ADMINISTRACION DE EMPRESAS.', 'ARTE DRAMÁTICO', 'ADMINISTRACION  DE EMPRESAS.'}
Realizando predicciones...
✓ Archivo de submission guardado en: /content/drive/MyDrive/IA/SaberPro